In [1]:
import sys
[sys.path.append(i) for i in ['.', '..']]
import torch
import numpy as np
import sigpy as sp
from sigpy.mri.app import L1WaveletRecon
from prox import Prox
from linear.linearmaps import LinearMap
from linear.mri import FFTn

In [2]:
from skimage.data import shepp_logan_phantom

In [3]:
ex_multi = np.load('AXT2_210_6001736_layer11.npz')

In [4]:
k_c = torch.tensor(ex_multi['k_r'] + 1j*ex_multi['k_i'])
s_c = torch.tensor(ex_multi['s_r'] + 1j*ex_multi['s_i'])
(nc, nh, nw) = s_c.shape

In [5]:
Fop = FFTn((nc, nh, nw), (nc, nh, nw), (1,2))

In [6]:
Fop(k_c)

tensor([[[ 11.2008-3.3383j, -10.2951-3.3605j,   2.7278-8.6322j,
           ...,  -5.9283+0.5132j,  -2.9312+1.9225j,
            3.2511+0.9525j],
         [ 11.7987-1.3793j,   9.6099+7.9255j,  -6.0592-5.1395j,
           ...,   4.1521+1.8373j,  -2.5432-3.7683j,
            1.5762+2.2252j],
         [  2.9915+11.0557j,   6.9505-0.9910j,  -4.7008+4.2421j,
           ...,  -2.5576+9.0741j,   3.8728-2.3565j,
            0.7245-2.0751j],
         ...,
         [ -1.0477-7.4192j,  -1.1894+6.6377j,  -0.3949-5.2509j,
           ...,  -3.5174-4.2480j,   6.0853+5.1471j,
            1.9705+7.5392j],
         [ -3.9844+0.9927j,   1.5184+2.1423j,   3.2092-11.0598j,
           ...,  -4.7167-0.5474j,  -6.1670-1.7790j,
            0.5188+1.2359j],
         [ -0.3102+6.4653j,   0.3453+3.0936j,  -0.2220-5.8773j,
           ...,  -6.1579+8.3253j,   2.1285+0.4830j,
            0.4567+3.6288j]],

        [[  2.6511+5.6158j,   7.1003-5.4108j,   3.6819+3.8053j,
           ...,  -5.1584-5.2030j,  -3.7312+2.375

In [7]:
Fop*k_c

tensor([[[ 11.2008-3.3383j, -10.2951-3.3605j,   2.7278-8.6322j,
           ...,  -5.9283+0.5132j,  -2.9312+1.9225j,
            3.2511+0.9525j],
         [ 11.7987-1.3793j,   9.6099+7.9255j,  -6.0592-5.1395j,
           ...,   4.1521+1.8373j,  -2.5432-3.7683j,
            1.5762+2.2252j],
         [  2.9915+11.0557j,   6.9505-0.9910j,  -4.7008+4.2421j,
           ...,  -2.5576+9.0741j,   3.8728-2.3565j,
            0.7245-2.0751j],
         ...,
         [ -1.0477-7.4192j,  -1.1894+6.6377j,  -0.3949-5.2509j,
           ...,  -3.5174-4.2480j,   6.0853+5.1471j,
            1.9705+7.5392j],
         [ -3.9844+0.9927j,   1.5184+2.1423j,   3.2092-11.0598j,
           ...,  -4.7167-0.5474j,  -6.1670-1.7790j,
            0.5188+1.2359j],
         [ -0.3102+6.4653j,   0.3453+3.0936j,  -0.2220-5.8773j,
           ...,  -6.1579+8.3253j,   2.1285+0.4830j,
            0.4567+3.6288j]],

        [[  2.6511+5.6158j,   7.1003-5.4108j,   3.6819+3.8053j,
           ...,  -5.1584-5.2030j,  -3.7312+2.375

In [8]:
Fop.H(k_c)

tensor([[[ 11.2008-3.3383j,   3.2511+0.9525j,  -2.9312+1.9225j,
           ...,   0.7571+7.7667j,   2.7278-8.6322j,
          -10.2951-3.3605j],
         [ -0.3102+6.4653j,   0.4567+3.6288j,   2.1285+0.4830j,
           ...,  -8.7947-3.6257j,  -0.2220-5.8773j,
            0.3453+3.0936j],
         [ -3.9844+0.9927j,   0.5188+1.2359j,  -6.1670-1.7790j,
           ...,  -1.4366+2.3460j,   3.2092-11.0598j,
            1.5184+2.1423j],
         ...,
         [  2.9267-1.8532j,   1.0965+2.7170j,  -5.7920-9.3521j,
           ...,   4.6212+5.7159j,   2.7798+4.3290j,
           -5.2340-2.9168j],
         [  2.9915+11.0557j,   0.7245-2.0751j,   3.8728-2.3565j,
           ...,   4.2110-3.8464j,  -4.7008+4.2421j,
            6.9505-0.9910j],
         [ 11.7987-1.3793j,   1.5762+2.2252j,  -2.5432-3.7683j,
           ...,  -5.1511-8.6463j,  -6.0592-5.1395j,
            9.6099+7.9255j]],

        [[  2.6511+5.6158j,  -0.3440+4.3041j,  -3.7312+2.3753j,
           ...,   6.1556+9.0082j,   3.6819+3.805

In [9]:
Fop.H*k_c

tensor([[[ 11.2008-3.3383j,   3.2511+0.9525j,  -2.9312+1.9225j,
           ...,   0.7571+7.7667j,   2.7278-8.6322j,
          -10.2951-3.3605j],
         [ -0.3102+6.4653j,   0.4567+3.6288j,   2.1285+0.4830j,
           ...,  -8.7947-3.6257j,  -0.2220-5.8773j,
            0.3453+3.0936j],
         [ -3.9844+0.9927j,   0.5188+1.2359j,  -6.1670-1.7790j,
           ...,  -1.4366+2.3460j,   3.2092-11.0598j,
            1.5184+2.1423j],
         ...,
         [  2.9267-1.8532j,   1.0965+2.7170j,  -5.7920-9.3521j,
           ...,   4.6212+5.7159j,   2.7798+4.3290j,
           -5.2340-2.9168j],
         [  2.9915+11.0557j,   0.7245-2.0751j,   3.8728-2.3565j,
           ...,   4.2110-3.8464j,  -4.7008+4.2421j,
            6.9505-0.9910j],
         [ 11.7987-1.3793j,   1.5762+2.2252j,  -2.5432-3.7683j,
           ...,  -5.1511-8.6463j,  -6.0592-5.1395j,
            9.6099+7.9255j]],

        [[  2.6511+5.6158j,  -0.3440+4.3041j,  -3.7312+2.3753j,
           ...,   6.1556+9.0082j,   3.6819+3.805

In [10]:
Fop.adjoint(k_c)

tensor([[[ 11.2008-3.3383j,   3.2511+0.9525j,  -2.9312+1.9225j,
           ...,   0.7571+7.7667j,   2.7278-8.6322j,
          -10.2951-3.3605j],
         [ -0.3102+6.4653j,   0.4567+3.6288j,   2.1285+0.4830j,
           ...,  -8.7947-3.6257j,  -0.2220-5.8773j,
            0.3453+3.0936j],
         [ -3.9844+0.9927j,   0.5188+1.2359j,  -6.1670-1.7790j,
           ...,  -1.4366+2.3460j,   3.2092-11.0598j,
            1.5184+2.1423j],
         ...,
         [  2.9267-1.8532j,   1.0965+2.7170j,  -5.7920-9.3521j,
           ...,   4.6212+5.7159j,   2.7798+4.3290j,
           -5.2340-2.9168j],
         [  2.9915+11.0557j,   0.7245-2.0751j,   3.8728-2.3565j,
           ...,   4.2110-3.8464j,  -4.7008+4.2421j,
            6.9505-0.9910j],
         [ 11.7987-1.3793j,   1.5762+2.2252j,  -2.5432-3.7683j,
           ...,  -5.1511-8.6463j,  -6.0592-5.1395j,
            9.6099+7.9255j]],

        [[  2.6511+5.6158j,  -0.3440+4.3041j,  -3.7312+2.3753j,
           ...,   6.1556+9.0082j,   3.6819+3.805

In [ ]:
# Ref recon with Sigpy
L1WaveletRecon(k_c, s_c, lamda = 1e-7).run()

In [11]:
k_c.requires_grad = True

In [15]:
a = torch.sum(torch.abs(Fop.H*k_c))

In [16]:
a.backward()

In [17]:
k_c.grad

tensor([[[ 0.0206-0.1692j, -0.9755+0.3083j,  0.6504-1.2840j,  ...,
           0.0407+0.4507j, -0.6233-0.3348j,  0.3041+0.3895j],
         [-0.2439+0.1771j, -0.2452+0.4463j,  0.0277+1.3658j,  ...,
           0.1368-0.2035j,  1.3528-0.8979j,  0.0547-0.5305j],
         [ 0.7384+1.3826j, -0.9679-0.6379j,  0.4586-1.5712j,  ...,
          -0.2686-0.2319j,  0.0285+0.0033j,  0.6709-0.2776j],
         ...,
         [-0.0965-0.3692j, -0.1101+0.9622j,  1.1616+0.5834j,  ...,
          -0.4523+0.1345j,  0.8275+1.0719j,  0.1432-0.5125j],
         [ 0.7464-0.1508j,  0.0365-0.6739j,  0.0504-1.3042j,  ...,
           1.0981+0.1884j, -0.8546+0.1110j, -0.5214+1.5475j],
         [ 0.1816+0.1374j, -0.6036-0.3452j,  0.2999-0.0338j,  ...,
          -1.2007+0.5064j,  0.2612-1.2401j,  0.1120-1.0085j]],

        [[-0.5053-0.6532j,  0.2175-0.3438j,  0.0134-0.1047j,  ...,
           0.7335-0.7939j,  0.6085+0.2870j, -0.8301-0.7802j],
         [ 0.0866-0.5146j, -0.0925+0.5601j,  0.3345-0.6217j,  ...,
           0.6

In [20]:
Fop.H*Fop*k_c

tensor([[[ 7.0000e+00-1.1000e+01j, -6.0000e+00+8.0000e+00j,
           9.0000e+00-9.0000e+00j,  ...,
           5.0000e+00+4.0000e+00j, -1.0000e+01-1.6000e+01j,
          -5.0000e+00-5.0000e+00j],
         [-3.0000e+00+8.0000e+00j,  1.0000e+00-2.6645e-14j,
           1.1000e+01+2.2000e+01j,  ...,
           8.0000e+00+2.0000e+00j,  2.0000e+01-2.6000e+01j,
          -8.0000e+00-2.8000e+01j],
         [ 9.0000e+00+2.1000e+01j, -1.0000e+01-2.0000e+00j,
           9.0000e+00-1.2000e+01j,  ...,
           2.0000e+00-7.0000e+00j,  9.0000e+00-2.1000e+01j,
           2.3000e+01-3.6000e+01j],
         ...,
         [ 1.0000e+01-1.0000e+01j, -7.0000e+00-1.0000e+00j,
           1.7000e+01+1.0000e+01j,  ...,
           8.0000e+00-9.0000e+00j, -1.0000e+00+1.2000e+01j,
           1.2000e+01-7.0000e+00j],
         [ 2.2000e+01-1.1000e+01j,  2.0000e+00-1.7000e+01j,
           4.0000e+00-1.9000e+01j,  ...,
           2.3000e+01-7.0000e+00j, -1.5000e+01-1.3000e+01j,
           3.0000e+00+1.4000e+01j],
 

In [21]:
(Fop.H*Fop)*k_c

tensor([[[ 7.0000e+00-1.1000e+01j, -6.0000e+00+8.0000e+00j,
           9.0000e+00-9.0000e+00j,  ...,
           5.0000e+00+4.0000e+00j, -1.0000e+01-1.6000e+01j,
          -5.0000e+00-5.0000e+00j],
         [-3.0000e+00+8.0000e+00j,  1.0000e+00-2.6645e-14j,
           1.1000e+01+2.2000e+01j,  ...,
           8.0000e+00+2.0000e+00j,  2.0000e+01-2.6000e+01j,
          -8.0000e+00-2.8000e+01j],
         [ 9.0000e+00+2.1000e+01j, -1.0000e+01-2.0000e+00j,
           9.0000e+00-1.2000e+01j,  ...,
           2.0000e+00-7.0000e+00j,  9.0000e+00-2.1000e+01j,
           2.3000e+01-3.6000e+01j],
         ...,
         [ 1.0000e+01-1.0000e+01j, -7.0000e+00-1.0000e+00j,
           1.7000e+01+1.0000e+01j,  ...,
           8.0000e+00-9.0000e+00j, -1.0000e+00+1.2000e+01j,
           1.2000e+01-7.0000e+00j],
         [ 2.2000e+01-1.1000e+01j,  2.0000e+00-1.7000e+01j,
           4.0000e+00-1.9000e+01j,  ...,
           2.3000e+01-7.0000e+00j, -1.5000e+01-1.3000e+01j,
           3.0000e+00+1.4000e+01j],
 